In [2]:
import pandas as pd
import sqlite3
from pybliometrics.scopus import AbstractRetrieval
from six import string_types, text_type
from crossref.restful import Works
# import ibis
import sys
sys.path.append("..")
from rcn_py import orcid
from rcn_py import database



    author-paper table
    author info table
    publication table

In [10]:
con = sqlite3.connect("tutorial.db")

In [11]:
cur = con.cursor()


In [12]:
cur.execute("CREATE TABLE test_paper(doi primary key, title, abstract)")

In [13]:
cur.execute("CREATE TABLE test_authors(orcid primary key, name, country)")


In [16]:
cur.execute("CREATE TABLE test_author_papers2(orcid, doi, primary key(orcid, doi) FOREIGN KEY(orcid) REFERENCES test_author(orcid), FOREIGN KEY(doi) REFERENCES test_paper(doi))")

In [55]:
con.commit()

In [56]:
con.close()

## Insert data

In [72]:
database.insert_database('',"Peter Kalverla")

'Done.'

In [ ]:
database.insert_coauthors_pub("Peter Kalverla")

In [ ]:
database.insert_cocoauthors("tutorial.db")

### Fetch data from database

In [ ]:
# Topic modeling (title or abstract) to cluster papers
clusters, idx2topics = database.pub_cluster(cur)

In [ ]:
# get coauthor relationship
links = database.fetch_relationships(cur)

In [ ]:
# assign author's group as the most frequent paper group
orcid_group = database.author_cluster(cur, clusters)

In [ ]:
# pyvis
database.build_network_database("Database")